In [10]:
import pyspark.sql.functions as F

In [2]:
%%html
<style>
table {float:left}
</style>

In [3]:
path2openAIRE = "/export/ml4ds/IntelComp/Datalake/openaire/20230328/parquet/"
path2S2 = "/export/ml4ds/IntelComp/Datalake/semanticscholar/20230418/parquet/"

# 1. Tables associated with Results (319 G)

## 1.1. Dataframe `result` (15 G)

This table contains `234.645.237` (no duplicates present)

<mark>Main conclusions:</mark>

   * <mark>Around 170 M publications</mark>
   * <mark>Language field is not relevant / normally not available</mark>
   * <mark>Reliable access right for around 110 M (out of which around 107 M publications)</mark>
   
Table structure (only most relevant fields are shown):

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| dateofcollection | string | Date when the result was collected by OpenAIRE |
| language | string | Language of the result (undetermined for most records) | 
| <mark>bestaccessrights</mark> | string | Open Access, Closed Access, etc ... |
| <mark>type</mark> | string | Type of the result |

Break down of results by `result type`, `language`, and `bestaccessright`

| Result Type | Number of entries | 
| --- | ---: | 
| publication| 169.889.028|
|    dataset| 57.765.051|
|      other|  6.682.075|
|   software|   309.083|

| Language | Number of entries | 
| --- | ---: | 
| Undetermined|174.583.154|
|           English| 28.030.135|
|           UNKNOWN|  4.756.037|
|            French|  4.294.807|
|            German|  3.459.609|
|           Russian|  2.824.807|
|          Japanese|  2.802.160|
|        Portuguese|  1.929.854|
|Spanish; Castilian|  1.815.234|
|           Italian|  1.558.993|
|           Turkish|  1.176.063|
|    Dutch; Flemish|  1.138.395|
|           Spanish|   879.085|
|          Croatian|   596.013|
|        Indonesian|   575.136|
|           Finnish|   443.164|
|         Ukrainian|   432.275|
|            Danish|   422.952|
|           Swedish|   382.228|
|             Czech|   367.393|

| Best Access Right | Number of entries | Number of entries (type = publication) |
| --- | ---: | ---: |
|       not available|117.709.144|62.608.424|
|         Open Access| 76.137.689|67.018.307|
|       Closed Access| 37.923.320|37.658.568|
|          Restricted|  2.625.938|2.454.176|
|              UNKOWN|    94.301|88.448|
|         Open Source|    83.824|0|
|             Embargo|    68.987|60.715|
|Creative Commons ...|      831|10|


In [7]:
results = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result").cache()
print('Number of results available:', results.count())
print('Number of results without duplicates:', results.dropDuplicates().count())
print('Number of results without duplicated ids:', results.dropDuplicates(["id"]).count())
results.printSchema()
results.show(n=2, truncate=120, vertical=True)

Number of results available: 234645237


Number of results without duplicates: 234645237


Number of results without duplicated ids: 234645237
root
 |-- id: string (nullable = true)
 |-- dateofcollection: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- bestaccessrights: string (nullable = true)
 |-- language: string (nullable = true)
 |-- dateofacceptance: string (nullable = true)
 |-- embargoenddate: string (nullable = true)
 |-- type: string (nullable = true)

-RECORD 0----------------------------------------------------------
 id               | openaire____::558701234257b3959ca02ea4f2ad5700 
 dateofcollection | 2023-03-22T21:54:13.595Z                       
 publisher        | null                                           
 journal          | null                                           
 bestaccessrights | Open Source                                    
 language         | Undetermined                                   
 dateofacceptance | null                                           
 embargoenddate   

In [13]:
results.groupBy("type").agg(F.count("*").alias("count")).show()
df = results.groupBy("language").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)
df = results.groupBy("bestaccessrights").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)

+-----------+---------+
|       type|    count|
+-----------+---------+
|    dataset| 57765051|
|   software|   309083|
|publication|169889028|
|      other|  6682075|
+-----------+---------+



+------------------+---------+
|          language|    count|
+------------------+---------+
|      Undetermined|174583154|
|           English| 28030135|
|           UNKNOWN|  4756037|
|            French|  4294807|
|            German|  3459609|
|           Russian|  2824807|
|          Japanese|  2802160|
|        Portuguese|  1929854|
|Spanish; Castilian|  1815234|
|           Italian|  1558993|
|           Turkish|  1176063|
|    Dutch; Flemish|  1138395|
|           Spanish|   879085|
|          Croatian|   596013|
|        Indonesian|   575136|
|           Finnish|   443164|
|         Ukrainian|   432275|
|            Danish|   422952|
|           Swedish|   382228|
|             Czech|   367393|
+------------------+---------+
only showing top 20 rows



+--------------------+---------+
|    bestaccessrights|    count|
+--------------------+---------+
|       not available|117709144|
|         Open Access| 76137689|
|       Closed Access| 37923320|
|          Restricted|  2625938|
|              UNKOWN|    94301|
|         Open Source|    83824|
|             Embargo|    68987|
|Creative Commons ...|      831|
|               Other|      212|
|     2013 CentERdata|       91|
|     2012 CentERdata|       79|
|     2014 CentERdata|       64|
|     2015 CentERdata|       59|
|     2010 CentERdata|       56|
|     2009 CentERdata|       49|
|     2011 CentERdata|       33|
|     2022 Centerdata|       20|
|Archived with tha...|       19|
|     2021 CentERdata|       16|
|     2019 CentERdata|       15|
+--------------------+---------+
only showing top 20 rows



In [14]:
results_journals = results.filter(results["type"] == "publication")
df = results_journals.groupBy("bestaccessrights").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)

+--------------------+--------+
|    bestaccessrights|   count|
+--------------------+--------+
|         Open Access|67018307|
|       not available|62608424|
|       Closed Access|37658568|
|          Restricted| 2454176|
|              UNKOWN|   88448|
|             Embargo|   60715|
|Archived with tha...|      19|
|CC BY-NC-ND 4.0 I...|      13|
|Creative Commons ...|      10|
|EM Publishers- Oi...|      10|
|               CC-BY|       8|
|This article is p...|       7|
|Creative Commons ...|       7|
|         CC BY-NC-ND|       7|
|The copyright hol...|       5|
|        CC-BY-SA 4.0|       5|
|Archived with tha...|       5|
|https://creativec...|       4|
|http://creativeco...|       4|
|         Open Source|       4|
+--------------------+--------+
only showing top 20 rows



## 1.2. Dataframe `result_titles` (25 G)

This table contains `346.862.861`. However:
   * We can remove duplicates and then we get `282.431.230`
   * The number of entries with different ids is just `234.644.580`. The reason is that some results have several titles, normally in different languages

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>Most results contain at least one title</mark>
   * <mark>It is important to remove duplicates before use. When doing so, we may want to keep English titles</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>title</mark> | string | Title of the result |

In [28]:
result_titles = spark.read.parquet("file:///export/data_ml4ds/IntelComp/Datasets/openaire/20230328/result_titles").cache()
print('Number of result_titles available:', result_titles.count())
print('Number of results_titles without duplicates:', result_titles.dropDuplicates().count())
print('Number of results_titles without duplicated ids:', result_titles.dropDuplicates(["id"]).count())
result_titles.printSchema()
result_titles.show(n=2, truncate=120, vertical=True)

23/07/01 19:54:26 WARN CacheManager: Asked to cache already cached data.


Number of result_titles available: 346862861


Number of results_titles without duplicates: 282431230


Number of results_titles without duplicated ids: 234644580
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)

-RECORD 0----------------------------------------------------
 id    | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed      
 title | Southern Cordillera magnetotelluric site MMTN82-007 
-RECORD 1----------------------------------------------------
 id    | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed      
 title | Southern Cordillera magnetotelluric site MMTN82-007 
only showing top 2 rows



In [25]:
# Contar las ocurrencias de cada valor en la columna "id"
counts = result_titles.groupBy("id").count()

# Filtrar las filas donde el conteo sea mayor que 1
duplicates = counts.filter(counts["count"] > 1)

# Unirse con el DataFrame original para obtener los ejemplos de filas duplicadas
result_titles.join(duplicates, "id").show(vertical=True, truncate=120)


-RECORD 0-----------------------------------------------------------------------------------------------------
 id    | 355e65625b88::00b0f47fe85c6d5819ea5a0cdf09199b                                                       
 title | Betoni on nanorakenne                                                                                
 count | 2                                                                                                    
-RECORD 1-----------------------------------------------------------------------------------------------------
 id    | 355e65625b88::00b0f47fe85c6d5819ea5a0cdf09199b                                                       
 title | Concrete is a nanostructured material                                                                
 count | 2                                                                                                    
-RECORD 2-----------------------------------------------------------------------------------------------------
 

In [26]:
result_titles = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_titles").dropDuplicates(["id"]).cache()

results.join(result_titles, on='id', how='inner').count()

234644580

## 1.3. Dataframe `result_subject` (33 G)

This table contains `1.808.759.738` keywords/subjects for up to `149.825.540` results. A small number of duplicates is present. In general, results can have more than one subject or keyword.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>The most used types are: MAG (900 M), keywords (780 M), FoS (73 M), Medical Subject (20 M), arXiv (9,5 M), ACM (6 M)</mark>
   * <mark>Problem with MAG is newest records will not have MAG; FoS only level one, same as Semantic Scholar?</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>type</mark> | string | Type of the subject (MAG, FoS, keyword ...) |
| <mark>subject</mark> | string | Value of the FoS, keyword, etc ... |

| Type | Number of entries |
| --- | ---: |
| Microsoft Academic Graph classification|899.629.544|
|                                 keyword|779.681.129|
|Fields of Science and Technology clas...| 73.263.133|
|                Medical Subject Headings| 20.160.013|
|    Library of Congress Subject Headings| 15.816.245|
|                                   arXiv|  9.536.185|
|     ACM Computing Classification System|  6.331.867|
|           Sustainable Development Goals|  1.451.227|
|            Dewey Decimal Classification|  1.265.389|
|                      JEL Classification|   849.784|

In [29]:
result_subject = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_subject").cache()
print('Number of result_subject available:', result_subject.count())
print('Number of results_subject without duplicates:', result_subject.dropDuplicates().count())
print('Number of results_subject without duplicated ids:', result_subject.dropDuplicates(["id"]).count())

23/07/01 19:55:57 WARN CacheManager: Asked to cache already cached data.


Number of result_subject available: 1808759738


23/07/01 20:07:29 ERROR TaskSchedulerImpl: Lost executor 4 on node15.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/01 20:07:29 WARN TaskSetManager: Lost task 8.0 in stage 187.0 (TID 15984) (node15.cluster.tsc.uc3m.es executor 4): ExecutorLostFailure (executor 4 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/01 20:07:29 WARN TaskSetManager: Lost task 38.0 in stage 187.0 (TID 16014) (node15.cluster.tsc.uc3m.es executor 4): ExecutorLostFailure (executor 4 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/01 20:07:29 WARN TaskSetManager: Lost task 28.0 in stage 187.0 (TID 16004) (node15.clus

Number of results_subject without duplicates: 1795482911


Number of results_subject without duplicated ids: 149825540
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)

-RECORD 0----------------------------------------------------
 id    | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed      
 title | Southern Cordillera magnetotelluric site MMTN82-007 
-RECORD 1----------------------------------------------------
 id    | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed      
 title | Southern Cordillera magnetotelluric site MMTN82-007 
only showing top 2 rows



In [30]:
result_subject.printSchema()
result_subject.show(n=2, truncate=120, vertical=True)

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- subject: string (nullable = true)

-RECORD 0-------------------------------------------------
 id      | 355e65625b88::5ae502516e23563158ec10bd919e2def 
 type    | keyword                                        
 subject | power plant                                    
-RECORD 1-------------------------------------------------
 id      | 355e65625b88::5ae502516e23563158ec10bd919e2def 
 type    | keyword                                        
 subject | maintenance                                    
only showing top 2 rows



In [35]:
df = result_subject.groupBy("type").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20, truncate=40)

+----------------------------------------+---------+
|                                    type|    count|
+----------------------------------------+---------+
| Microsoft Academic Graph classification|899629544|
|                                 keyword|779681129|
|Fields of Science and Technology clas...| 73263133|
|                Medical Subject Headings| 20160013|
|    Library of Congress Subject Headings| 15816245|
|                                   arXiv|  9536185|
|     ACM Computing Classification System|  6331867|
|           Sustainable Development Goals|  1451227|
|            Dewey Decimal Classification|  1265389|
|                      JEL Classification|   849784|
|           Nippon Decimal Classification|   507120|
|        Universal Decimal Classification|   165345|
|      Mathematics Subject Classification|    72863|
|    National Diet Library Classification|     6252|
|                                 EuroVoc|     6039|
|                           EU Data Theme|    

## 1.4. Dataframe `result_relevantdate` (6,8 G)

This table contains `322.980.801` dates for up to `142.186.145` results. A small number of duplicates is present. In general, results can have more than one relevant date.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>Not sure what is the best date to use, maybe published-print or published-online</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>type</mark> | string | Type of Date reported (created, published-print, published-online, submitted, etc)|
| <mark>subject</mark> | string | **Date - Careful !! the name of the field does not seem correct !**. |

Types of dates that are present in the dataset:

| Type of date | Number of entries |
| --- | ---: |
| created|121.753.738|
| published-print| 94.781.868|
|published-online| 46.875.672|
|          issued| 26.997.996|
|         updated| 11.153.826|
|       available|  9.777.723|
|         UNKNOWN|  4.518.579|
|       submitted|  3.957.615|
|        accepted|  1.369.328|
|        Accepted|  1.041.690|
|       collected|   644.163|
|     copyrighted|    61.196|
|           valid|    29.254|
|       withdrawn|    11.946|
|           other|     6.195|
|         endDate|        6|
|       startDate|        6|

In [36]:
result_relevantdate = spark.read.parquet("file:///export/data_ml4ds/IntelComp/Datasets/openaire/20230328/result_relevantdate").cache()
print('Number of result_relevantdate available:', result_relevantdate.count())
print('Number of results_relevantdate without duplicates:', result_relevantdate.dropDuplicates().count())
print('Number of results_relevantdate without duplicated ids:', result_relevantdate.dropDuplicates(["id"]).count())
result_relevantdate.printSchema()
result_relevantdate.show(n=2, truncate=120, vertical=True)

Number of result_relevantdate available: 322980801


Number of results_relevantdate without duplicates: 322880780


Number of results_relevantdate without duplicated ids: 142186145
root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- subject: string (nullable = true)

-RECORD 0-------------------------------------------------
 id      | 57a035e5b1ae::03a709c6d2e90d0168ab01b6cd93757b 
 type    | issued                                         
 subject | 2006-01-01                                     
-RECORD 1-------------------------------------------------
 id      | 57a035e5b1ae::03e86c991b36f79bb4c31643d368b52c 
 type    | issued                                         
 subject | 2014-01-01                                     
only showing top 2 rows



In [38]:
df = result_relevantdate.groupBy("type").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20, truncate=40)

+----------------+---------+
|            type|    count|
+----------------+---------+
|         created|121753738|
| published-print| 94781868|
|published-online| 46875672|
|          issued| 26997996|
|         updated| 11153826|
|       available|  9777723|
|         UNKNOWN|  4518579|
|       submitted|  3957615|
|        accepted|  1369328|
|        Accepted|  1041690|
|       collected|   644163|
|     copyrighted|    61196|
|           valid|    29254|
|       withdrawn|    11946|
|           other|     6195|
|         endDate|        6|
|       startDate|        6|
+----------------+---------+



## 1.5. Dataframe `result_project` (374 M)

This table contains `5.386.150` project acknowledgments for up to `3.435.844` results. No duplicates are present. In general, results can have more than one project they acknowledge. In total, there are results for `748.412` projects.

A few entries of this table (around `200.000`) cannot be linked to results using the results `id`.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>Acknowlegments are available for the following funders: Corda_h2020 (1.155.065), nih (1 M), nsf (1M), corda(500 K). Others less represented.</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>project</mark> | string | Unique identifier of the linked project. Format starts with funding source (e.g., corda_h2020, nsf, etc) |

| Project Type | Number of entries | 
| --- | ---: |
|corda__h2020|1.155.065|
|nih_________| 983.661|
|nsf_________| 965.101|
|corda_______| 506.452|
|nserc_______| 233.810|
|wt__________| 203.759|
|fct_________| 162.358|
|anr_________| 161.389|
|ukri________| 145.809|
|snsf________| 132.436|
|arc_________| 100.880|
|cihr________|  96.940|
|mestd_______|  77.829|
|aka_________|  68.985|
|nwo_________|  50.271|
|nhmrc_______|  50.158|
|rsf_________|  49.566|
|miur________|  44.808|
|fwf_________|  39.173|
|sfi_________|  28.944|

In [40]:
result_project = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_project").cache()
print('Number of result_project available:', result_project.count())
print('Number of results_project without duplicates:', result_project.dropDuplicates().count())
print('Number of results_project without duplicated ids:', result_project.dropDuplicates(["id"]).count())
result_project.printSchema()
result_project.show(n=50, truncate=120, vertical=True)

23/07/01 21:09:54 WARN CacheManager: Asked to cache already cached data.


Number of result_project available: 5386150


Number of results_project without duplicates: 5386150


Number of results_project without duplicated ids: 3435844
root
 |-- id: string (nullable = true)
 |-- project: string (nullable = true)

-RECORD 0-------------------------------------------------
 id      | 57a035e5b1ae::1008bd45199be35e79f88082cf6f41c2 
 project | irb_hr______::2669e0d43c4213e26d8aa1534c58b30b 
-RECORD 1-------------------------------------------------
 id      | 57a035e5b1ae::a3264873505e971859d36093dc6cf337 
 project | irb_hr______::a3eeae587bcddb80a0f5f3cbb134ea6b 
-RECORD 2-------------------------------------------------
 id      | arXiv_______::05d02f08278e1ad4ac0548780244a481 
 project | corda__h2020::985a86ddffa25cacf60d38af5634c509 
-RECORD 3-------------------------------------------------
 id      | arXiv_______::407dad8c044f988d4592172368424d72 
 project | corda_______::f422e97791e34995d54bf773dfb2007d 
-RECORD 4-------------------------------------------------
 id      | arXiv_______::8663d1881255a8697331d8a7c677c88f 
 project | corda__h2020::40510353be8b

In [41]:
print('Number of acknowledged projects:', result_project.dropDuplicates(["project"]).count())

Number of acknowledged projects: 748412


In [42]:
results.join(result_project, on='id', how='inner').count()

5184693

In [53]:
from pyspark.sql.types import StringType

get_funder_udf = F.udf(lambda x: x.split(':')[0], StringType())

result_funder = result_project.withColumn("Funder", get_funder_udf(F.col("project")))

df = result_funder.groupBy("Funder").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)


23/07/02 00:00:29 ERROR TaskSchedulerImpl: Lost executor 7 on node56.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:00:29 WARN TaskSetManager: Lost task 16.0 in stage 284.0 (TID 24785) (node56.cluster.tsc.uc3m.es executor 7): ExecutorLostFailure (executor 7 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:00:29 WARN TaskSetManager: Lost task 39.0 in stage 284.0 (TID 24803) (node56.cluster.tsc.uc3m.es executor 7): ExecutorLostFailure (executor 7 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:00:29 WARN TaskSetManager: Lost task 4.0 in stage 284.0 (TID 24775) (node56.clus

+------------+-------+
|      Funder|  count|
+------------+-------+
|corda__h2020|1155065|
|nih_________| 983661|
|nsf_________| 965101|
|corda_______| 506452|
|nserc_______| 233810|
|wt__________| 203759|
|fct_________| 162358|
|anr_________| 161389|
|ukri________| 145809|
|snsf________| 132436|
|arc_________| 100880|
|cihr________|  96940|
|mestd_______|  77829|
|aka_________|  68985|
|nwo_________|  50271|
|nhmrc_______|  50158|
|rsf_________|  49566|
|miur________|  44808|
|fwf_________|  39173|
|sfi_________|  28944|
+------------+-------+
only showing top 20 rows



## 1.6. Dataframe `result_description` (104 G)

This table contains `135.019.530` descriptions for up to `120.876.911` results. A small number of duplicates is present (around 19K). In general, results can have more than one description.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>If we restrict ourselves to `results` of the type `publication` there are `111.771.100` descriptions available, for `102.097.393` different `results`.</mark>
   * <mark>Most of the descriptions in this case are abstracts, but we should be careful because some of them look erroneous and too short.</mark>
   * <mark>There are also some descriptions in a variety of languages (though most seem to be in English)</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>description</mark> | string | Description |

In [54]:
result_description = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_description").cache()
print('Number of result_description available:', result_description.count())
print('Number of results_description without duplicates:', result_description.dropDuplicates().count())
print('Number of results_description without duplicated ids:', result_description.dropDuplicates(["id"]).count())
result_description.printSchema()
result_description.show(n=50, truncate=120, vertical=True)

Number of result_description available: 135019530


23/07/02 00:22:54 ERROR TaskSchedulerImpl: Lost executor 17 on node38.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:22:54 WARN TaskSetManager: Lost task 18.0 in stage 293.0 (TID 27994) (node38.cluster.tsc.uc3m.es executor 17): ExecutorLostFailure (executor 17 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:22:54 WARN TaskSetManager: Lost task 38.0 in stage 293.0 (TID 28014) (node38.cluster.tsc.uc3m.es executor 17): ExecutorLostFailure (executor 17 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:22:54 WARN TaskSetManager: Lost task 8.0 in stage 293.0 (TID 27984) (node38

Number of results_description without duplicates: 135000638


23/07/02 00:32:20 ERROR TaskSchedulerImpl: Lost executor 14 on node19.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:32:20 WARN TaskSetManager: Lost task 7.0 in stage 299.0 (TID 29776) (node19.cluster.tsc.uc3m.es executor 14): ExecutorLostFailure (executor 14 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:32:20 WARN TaskSetManager: Lost task 37.0 in stage 299.0 (TID 29806) (node19.cluster.tsc.uc3m.es executor 14): ExecutorLostFailure (executor 14 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:32:20 WARN TaskSetManager: Lost task 27.0 in stage 299.0 (TID 29796) (node19

Number of results_description without duplicated ids: 120876911
root
 |-- id: string (nullable = true)
 |-- description: string (nullable = true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------
 id          | 475c1990cbb2::027ec82d0a07db1ccb816f1d22c805a9                                                                           
 description | Using both satellite and in situ data sets, this project will document the seasonal (daily to monthly timescales) of ... 
-RECORD 1-------------------------------------------------------------------------------------------------------------------------------
 id          | 475c1990cbb2::11877eba3e6c37340c7a04966cf4fc75                                                                           
 description | <p>The correspondence files contain unique identifiers for the 1986 census geographic area and the corresponding uniq... 
-RECORD 2----------------------

In [62]:
aux_df = results.filter(results["type"] == "publication").join(result_description, "id").cache()
aux_df.show(vertical=True, truncate=120)

23/07/02 18:24:38 ERROR TaskSchedulerImpl: Lost executor 21 on node06.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:24:38 WARN TaskSetManager: Lost task 1003.0 in stage 393.0 (TID 39193) (node06.cluster.tsc.uc3m.es executor 21): ExecutorLostFailure (executor 21 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:24:38 WARN TaskSetManager: Lost task 999.0 in stage 393.0 (TID 39192) (node06.cluster.tsc.uc3m.es executor 21): ExecutorLostFailure (executor 21 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:24:38 WARN TaskSetManager: Lost task 915.0 in stage 393.0 (TID 39168) (n

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------
 id               | 07b5c0ccd4fe::9f9724ac702086fe401db82ca2fa2726                                                                           
 dateofcollection | 2023-03-14T08:31:35.406Z                                                                                                 
 publisher        | RS Global Sp. z O.O.                                                                                                     
 journal          | World Science                                                                                                            
 bestaccessrights | Open Access                                                                                                              
 language         | English                                                                                                                  
 dateo

In [64]:
aux_df.count()

23/07/02 18:40:30 ERROR TaskSchedulerImpl: Lost executor 27 on node93.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:40:30 WARN TaskSetManager: Lost task 9.0 in stage 397.0 (TID 40408) (node93.cluster.tsc.uc3m.es executor 27): ExecutorLostFailure (executor 27 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:40:30 WARN TaskSetManager: Lost task 29.0 in stage 397.0 (TID 40428) (node93.cluster.tsc.uc3m.es executor 27): ExecutorLostFailure (executor 27 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 18:40:30 WARN TaskSetManager: Lost task 38.0 in stage 397.0 (TID 40437) (node93

111771100

In [65]:
aux_df.dropDuplicates(["id"]).count()

102097393

## 1.7. Dataframe `result_citations` (52 G)

This table contains `1.277.513.949` citations. No duplicates are present. In total, there are citations for `59.339.266` results.

As a reference, the number of citations in Semantic Scholar is approximately double (255 M)

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>The vast majority (`1.227.235.601`) are citations for publications</mark>
   * <mark>Take into account the difference in numbers with respect to semantic Scholar. Results will not be comparable</mark>.

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>target</mark> | string | Unique identifier of the cited result |

In [55]:
result_citations = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_citations").cache()
print('Number of result_citations available:', result_citations.count())
print('Number of results_citations without duplicates:', result_citations.dropDuplicates().count())
print('Number of results_citations without duplicated ids:', result_citations.dropDuplicates(["id"]).count())
result_citations.printSchema()
result_citations.show(n=50, truncate=120, vertical=True)

Number of result_citations available: 1277513949


23/07/02 00:41:06 ERROR TaskSchedulerImpl: Lost executor 15 on node56.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:41:06 WARN TaskSetManager: Lost task 492.0 in stage 308.0 (TID 31386) (node56.cluster.tsc.uc3m.es executor 15): ExecutorLostFailure (executor 15 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:41:06 WARN TaskSetManager: Lost task 501.0 in stage 308.0 (TID 31389) (node56.cluster.tsc.uc3m.es executor 15): ExecutorLostFailure (executor 15 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 00:41:06 WARN TaskSetManager: Lost task 543.0 in stage 308.0 (TID 31394) (no

Number of results_citations without duplicates: 1277513949


Number of results_citations without duplicated ids: 59339266
root
 |-- id: string (nullable = true)
 |-- target: string (nullable = true)

-RECORD 0------------------------------------------------
 id     | doi_________::0000bb1da39af46c1c64dc11ac1a5b17 
 target | doi_________::8bf41e24e45eb75adbe5fc8a55a6f566 
-RECORD 1------------------------------------------------
 id     | doi_________::0000ed1830f939df304f965aeb01ff44 
 target | doi_________::32f38d2794ebcef3704ece65f6eebc2f 
-RECORD 2------------------------------------------------
 id     | doi_________::000228af67c92f29dac75d5be146cb65 
 target | doi_dedup___::44a2694d6305dd47b160ae80c930dc8f 
-RECORD 3------------------------------------------------
 id     | doi_________::000228af67c92f29dac75d5be146cb65 
 target | doi_dedup___::2b824ba8e30474a57cbf6a59430ee159 
-RECORD 4------------------------------------------------
 id     | doi_________::000228af67c92f29dac75d5be146cb65 
 target | doi_dedup___::af10e32eacc2f41eea7914444

In [66]:
aux_df = results.filter(results["type"] == "publication").join(result_citations, "id").cache()
aux_df.count()

23/07/02 19:04:27 ERROR TaskSchedulerImpl: Lost executor 26 on node31.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 19:04:27 WARN TaskSetManager: Lost task 28.0 in stage 411.0 (TID 41874) (node31.cluster.tsc.uc3m.es executor 26): ExecutorLostFailure (executor 26 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 19:04:27 WARN TaskSetManager: Lost task 18.0 in stage 411.0 (TID 41864) (node31.cluster.tsc.uc3m.es executor 26): ExecutorLostFailure (executor 26 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/07/02 19:04:27 WARN TaskSetManager: Lost task 38.0 in stage 411.0 (TID 41884) (node3

1277235601

## 1.8. Dataframe `result_instance` (32 G)

Expand this section to see the structure of this table. In principle, I believe it is not relevant for us, since the relevant information can also be obtained from other tables.

In [56]:
result_instance = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_instance").cache()
print('Number of result_instance available:', result_instance.count())
print('Number of results_instance without duplicates:', result_instance.dropDuplicates().count())
print('Number of results_instance without duplicated ids:', result_instance.dropDuplicates(["id"]).count())
result_instance.printSchema()
result_instance.show(n=50, truncate=120, vertical=True)

Number of result_instance available: 349393435


Number of results_instance without duplicates: 340921525


Number of results_instance without duplicated ids: 339481321
root
 |-- id: string (nullable = true)
 |-- result: string (nullable = true)
 |-- collectedfrom: string (nullable = true)
 |-- hostedby: string (nullable = true)
 |-- accessrights: string (nullable = true)
 |-- license: string (nullable = true)
 |-- peerreviewed: string (nullable = true)
 |-- type: string (nullable = true)
 |-- apc: string (nullable = true)
 |-- apccurrency: string (nullable = true)

-RECORD 0-----------------------------------------------------------------------------------------
 id            | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed::9ce697a2118a0dcdda1e1a0727e381e9 
 result        | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed                                   
 collectedfrom | openaire____::3795d6478e30e2c9f787d427ff160944                                   
 hostedby      | openaire____::3795d6478e30e2c9f787d427ff160944                                   
 accessrights  | Open Access            

## 1.9. Dataframe `result_instance_pid` (27 G)

<mark>This table contains identifiers for publications and other results. It is a key table, as it allows to merge OpenAIRE with other data sources.</mark>

Currently, there are `283.365.955` identifiers, with a very small number of duplicates. Some results have more than one identifier.


<mark>Main conclusions:</mark>

   * <mark>Make sure to use the `result` field for joining this table with others</mark>
   * <mark>Regarding publications, there are 156 M DOIs, 40,5 M PubMed Id</mark>

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| id | string | Identifier of the result |
| <mark>result</mark> | string | Unique identifier of the result in the OpenAIRE graph. **This is the one that can be used for merging operations, i.e., similar to id in other tables** |
| <mark>type</mark> | string | Tipe of identifer contained (e.g. Doi, pubmed, etc.) |
| <mark>pid</mark> | string | Value of the doi, pubmed_id, etc. |

Split of external ids by type
| Type | Number of entries | Number of entries for publications |
| --- | ---: | ---: |
|Digital Object Identifier|176.527.468|156.041.904|
|           PubMed ID| 40.506.906|40.497.136|
|             uniprot| 355.059.65||
|   PubMed Central ID| 15.361.161|15.361.053|
|              Handle| 10.433.864||
|                 ena|  2.388.733||
|               arXiv|  2.219.416|2.219.416|
|Protein Data Bank...|   419.147||
|                w3id|     3.295||

In [57]:
result_instance_pid = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_instance_pid").cache()
print('Number of result_instance_pid available:', result_instance_pid.count())
print('Number of results_instance_pid without duplicates:', result_instance_pid.dropDuplicates().count())
print('Number of results_instance_pid without duplicated ids:', result_instance_pid.dropDuplicates(["id"]).count())
result_instance_pid.printSchema()
result_instance_pid.show(n=50, truncate=120, vertical=True)

Number of result_instance_pid available: 283365955


Number of results_instance_pid without duplicates: 283083610


Number of results_instance_pid without duplicated ids: 262401849
root
 |-- id: string (nullable = true)
 |-- result: string (nullable = true)
 |-- type: string (nullable = true)
 |-- pid: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------
 id     | dedup_wf_001::08cce4e3aa27c273045bd10f8cd6c5e3::6a96ed22726abfc9a4aa6d16f05842d3 
 result | dedup_wf_001::08cce4e3aa27c273045bd10f8cd6c5e3                                   
 type   | Handle                                                                           
 pid    | 10261/68199                                                                      
-RECORD 1----------------------------------------------------------------------------------
 id     | dedup_wf_001::db18bb19ca28b11d11bb02e00b3b32d5::9f337ce3b28c84bdc6fe6d13e686ac34 
 result | dedup_wf_001::db18bb19ca28b11d11bb02e00b3b32d5                                   
 type   | Handle                                  

In [68]:
df = result_instance_pid.groupBy("type").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)

+--------------------+---------+
|                type|    count|
+--------------------+---------+
|Digital Object Id...|176527468|
|           PubMed ID| 40506906|
|             uniprot| 35505965|
|   PubMed Central ID| 15361161|
|              Handle| 10433864|
|                 ena|  2388733|
|               arXiv|  2219416|
|Protein Data Bank...|   419147|
|                w3id|     3295|
+--------------------+---------+



In [71]:
results.join(result_instance_pid, results["id"] == result_instance_pid["result"]).count()

283365955

In [76]:
results.withColumnRenamed("type", "resultType").join(result_instance_pid, results["id"] == result_instance_pid["result"]).show(n=2)

+--------------------+--------------------+---------+-------+----------------+--------+----------------+--------------+----------+--------------------+--------------------+------+-----------+
|                  id|    dateofcollection|publisher|journal|bestaccessrights|language|dateofacceptance|embargoenddate|resultType|                  id|              result|  type|        pid|
+--------------------+--------------------+---------+-------+----------------+--------+----------------+--------------+----------+--------------------+--------------------+------+-----------+
|73490d0e0f82::043...|2023-03-13T01:12:...|     null|   null|   Closed Access| English|      2006-09-01|          null|     other|73490d0e0f82::043...|73490d0e0f82::043...|Handle|11375/21259|
|73490d0e0f82::05a...|2023-03-13T01:12:...|     null|   null|   Closed Access| UNKNOWN|      1986-06-01|          null|     other|73490d0e0f82::05a...|73490d0e0f82::05a...|Handle|11375/14225|
+--------------------+------------------

In [82]:
df_aux2 = df_aux.filter(df_aux["resultType"] == "publication").groupBy("type").agg(F.count("*").alias("count"))
df_aux2.orderBy(df_aux2["count"].desc()).show(20)

+--------------------+---------+
|                type|    count|
+--------------------+---------+
|Digital Object Id...|156041904|
|           PubMed ID| 40497136|
|   PubMed Central ID| 15361053|
|              Handle|  8809478|
|               arXiv|  2219416|
+--------------------+---------+



### 1.9.a. Number of unique DOIs and PMIDs

In [13]:
result_instance_pid = spark.read.parquet(path2openAIRE + "result_instance_pid").cache()
result_doi = result_instance_pid.filter(result_instance_pid["type"] == "Digital Object Identifier").cache()
print("Entradas de tipo DOI:", result_doi.count())
col_to_keep = ["result", "pid"]
result_doi = result_doi.select(*col_to_keep)
print("Entradas únicas:", result_doi.dropDuplicates().count())
result_doi = result_doi.select("pid").dropDuplicates().cache()
print("DOIs únicos", result_doi.count())

23/08/28 20:06:12 WARN CacheManager: Asked to cache already cached data.        
23/08/28 20:06:12 WARN CacheManager: Asked to cache already cached data.


Entradas de tipo DOI: 176527468


Entradas únicas: 146888060


DOIs únicos 146886438


In [14]:
result_instance_pid = spark.read.parquet(path2openAIRE + "result_instance_pid").cache()
result_pmid = result_instance_pid.filter(result_instance_pid["type"] == "PubMed ID").cache()
print("Entradas de tipo PubMed ID:", result_pmid.count())
col_to_keep = ["result", "pid"]
result_pmid = result_pmid.select(*col_to_keep)
print("Entradas únicas:", result_pmid.dropDuplicates().count())
result_pmid = result_pmid.select("pid").dropDuplicates().cache()
print("PubMed ID únicos", result_pmid.count())

23/08/28 20:07:38 WARN CacheManager: Asked to cache already cached data.
23/08/28 20:07:38 WARN CacheManager: Asked to cache already cached data.


Entradas de tipo PubMed ID: 40506906


Entradas únicas: 33939208


PubMed ID únicos 33939208


### 1.9.b. Merging with Semantic Scholar

We are going to write some code and identify how many publications can be matched between Semantic Scholar and OpenAIRE using DOI and/or PubMed identifiers

In [15]:
papers_S2 = spark.read.parquet(path2S2 + "papers.parquet").cache()
papers_S2.show(n=5, truncate=120, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------
 id                       | 26                                                                                                                       
 title                    | FPGA-based design and implementation of an approximate polynomial matrix EVD algorithm                                   
 S2Url                    | https://www.semanticscholar.org/paper/7011b84b03f1d992962c4a6c87459f7742bc3165                                           
 year                     | 2012                                                                                                                     
 doi                      | 10.1109/FPT.2012.6412125                                                                                                 
 pmid                     | null                                                                    

In [16]:
doi_matches = result_doi.join(papers_S2, result_doi.pid == papers_S2.doi, how='inner').cache()
pmid_matches = result_pmid.join(papers_S2, result_pmid.pid == papers_S2.pmid, how='inner').cache()

In [17]:
print("Number of matches using DOI:", doi_matches.count())
print("Number of matches using PMID:", pmid_matches.count())

23/08/28 20:30:40 ERROR TaskSchedulerImpl: Lost executor 7 on node38.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:30:40 WARN TaskSetManager: Lost task 129.0 in stage 129.0 (TID 13455) (node38.cluster.tsc.uc3m.es executor 7): ExecutorLostFailure (executor 7 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:30:40 WARN TaskSetManager: Lost task 137.0 in stage 129.0 (TID 13467) (node38.cluster.tsc.uc3m.es executor 7): ExecutorLostFailure (executor 7 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:30:40 WARN TaskSetManager: Lost task 122.0 in stage 129.0 (TID 13452) (node38.

Number of matches using DOI: 59451782


23/08/28 20:39:37 ERROR TaskSchedulerImpl: Lost executor 1 on node93.cluster.tsc.uc3m.es: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:39:37 WARN TaskSetManager: Lost task 320.0 in stage 135.0 (TID 13971) (node93.cluster.tsc.uc3m.es executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:39:37 WARN TaskSetManager: Lost task 349.0 in stage 135.0 (TID 13974) (node93.cluster.tsc.uc3m.es executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/08/28 20:39:37 WARN TaskSetManager: Lost task 348.0 in stage 135.0 (TID 13973) (node93.

Number of matches using PMID: 33791420


## 1.10. Dataframe `result_author` (18 G)

This table contains `677.553.498` authorships for up to `182.593.815` results. A small number of duplicates are present. In general, results can have more than author.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>Probably not very useful, as the author is identified with full name, making it diffult to join with other tables</mark>
   * <mark>Many errors seem to be present in the table, probably it implies that we should clean the table before use</mark>

Table structure (only most relevant fields are shown):

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>fullname</mark> | string | Full Name of the author |
| <mark>rank</mark> | integer | Order of the author in the list of authors |

In [58]:
result_author = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_author").cache()
print('Number of result_author available:', result_author.count())
print('Number of results_author without duplicates:', result_author.dropDuplicates().count())
print('Number of results_author without duplicated ids:', result_author.dropDuplicates(["id"]).count())
result_author.printSchema()
result_author.show(n=50, truncate=120, vertical=True)

Number of result_author available: 677553498


Number of results_author without duplicates: 670416829


Number of results_author without duplicated ids: 182593815
root
 |-- id: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- rank: integer (nullable = true)

-RECORD 0--------------------------------------------------------------------
 id       | 475c1990cbb2::03d8c172942128d4afda0f422ee02aed                   
 fullname | Natural Resources Canada | Ressources naturelles Canada          
 rank     | 1                                                                
-RECORD 1--------------------------------------------------------------------
 id       | 475c1990cbb2::159217f069786826c0cb6d0d00fd51fe                   
 fullname | Natural Resources Canada | Ressources naturelles Canada          
 rank     | 1                                                                
-RECORD 2--------------------------------------------------------------------
 id       | 475c1990cbb2::19b8e6021fbb7f7d93036e70a0479552                   
 fullname | Digital Chart of the World      

In [84]:
results.join(result_author, "id").count()

677553498

## 1.11. Dataframe `result_affiliation` (6.1 G)

This table contains `110.241.706` affiliations to organizations for up to `67.180.335` results. No duplicates are present. In general, results can have more than one organization in the list of affiliations.

<mark>Main conclusions:</mark>

   * <mark>Consistent table to be used with `result`</mark>
   * <mark>Can easily be used with tables associated to organizations for identifying organizations with activity in certain areas.</mark>

Table structure (only most relevant fields are shown):

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>organization</mark> | string | Unique identifier of the linked organization. Contains identifiers, not organization names |

In [59]:
result_affiliation = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/result_affiliation").cache()
print('Number of result_affiliation available:', result_affiliation.count())
print('Number of results_affiliation without duplicates:', result_affiliation.dropDuplicates().count())
print('Number of results_affiliation without duplicated ids:', result_affiliation.dropDuplicates(["id"]).count())
result_affiliation.printSchema()
result_affiliation.show(n=50, truncate=120, vertical=True)

Number of result_affiliation available: 110241706


Number of results_affiliation without duplicates: 110241706


Number of results_affiliation without duplicated ids: 67180335
root
 |-- id: string (nullable = true)
 |-- organization: string (nullable = true)

-RECORD 0------------------------------------------------------
 id           | arXiv_______::0d0195a68f0a5247ff3edc4c0c97da5a 
 organization | openorgs____::f19bd4f95b81943353a480436ef88fd0 
-RECORD 1------------------------------------------------------
 id           | arXiv_______::0e1493b9e93429b73e01e75c5e76f159 
 organization | openorgs____::f1c2611f05338200488d25e838c518d5 
-RECORD 2------------------------------------------------------
 id           | arXiv_______::111398dc0e1a4d4c366b9a02964ee6a9 
 organization | pending_org_::63d0c85afe462ef8dc71bd818ec294a2 
-RECORD 3------------------------------------------------------
 id           | arXiv_______::153aabd37713a7ecb814a60c4ee34912 
 organization | pending_org_::e4e3f394d6d6fda50c450d4dbefd2fb1 
-RECORD 4------------------------------------------------------
 id           | arXiv

23/07/02 13:16:36 WARN TransportChannelHandler: Exception in connection from node21.cluster.tsc.uc3m.es/10.0.13.41:7337
java.io.IOException: Connection reset by peer
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:259)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEvent

In [85]:
results.join(result_affiliation, "id").count()

110240804

# 2. Tables associated with Organizations

## 2.1. Dataframe `organization` (27 M)

This table contains `314.059` organizations. No duplicates are present.

<mark>Main conclusions:</mark>

   * <mark>This table can be used togetter with `result_affiliation` to link results to organizations and/or countries</mark>
   * <mark>Number of organizations: US (54 K); GB (17,5 K), DE (13 K); ES (8,4 K); GR (3,9 K)</mark>

Table structure (only most relevant fields are shown):

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the organization in the OpenAIRE graph (can be joined with table `result_affiliation`)|
| <mark>name</mark> | string | Name of the organization |
| <mark>country</mark> | string | Country of the organization (expand group to see number of organizations per country) |

| Country | Number of organizations | 
| --- | ---:|
|UNKNOWN|92607|
|     US|54044|
|     GB|17659|
|     DE|13893|
|     FR|10569|
|     CH| 9504|
|     IT| 8515|
|     ES| 8383|
|     NL| 5701|
|     CN| 5526|
|     CA| 4648|
|     JP| 4502|
|     BE| 4191|
|     IN| 4121|
|     GR| 3881|
|     CZ| 3684|
|     SE| 3440|
|     NO| 3374|
|     PL| 3285|
|     RU| 2961|


In [87]:
organization = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/organization").cache()
print('Number of organization available:', organization.count())
print('Number of organization without duplicates:', organization.dropDuplicates().count())
print('Number of organization without duplicated ids:', organization.dropDuplicates(["id"]).count())
organization.printSchema()
organization.show(n=10, truncate=120, vertical=True)

23/07/02 23:00:42 WARN CacheManager: Asked to cache already cached data.


Number of organization available: 314059
Number of organization without duplicates: 314059
Number of organization without duplicated ids: 314059
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- legalshortname: string (nullable = true)
 |-- country: string (nullable = true)

-RECORD 0---------------------------------------------------------------------------
 id             | anr_________::1c325fed2fc40eeeeeffbfbc4f4e5f4d                    
 name           | Karlsruhe Institute of Technology - Allemagne                     
 legalshortname | null                                                              
 country        | UNKNOWN                                                           
-RECORD 1---------------------------------------------------------------------------
 id             | anr_________::bb8d415ee418e98e058e936192ec53e8                    
 name           | Physiologie des Adaptations Nutritionnelles                       
 legalshortnam

In [91]:
df = organization.groupBy("country").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)

+-------+-----+
|country|count|
+-------+-----+
|UNKNOWN|92607|
|     US|54044|
|     GB|17659|
|     DE|13893|
|     FR|10569|
|     CH| 9504|
|     IT| 8515|
|     ES| 8383|
|     NL| 5701|
|     CN| 5526|
|     CA| 4648|
|     JP| 4502|
|     BE| 4191|
|     IN| 4121|
|     GR| 3881|
|     CZ| 3684|
|     SE| 3440|
|     NO| 3374|
|     PL| 3285|
|     RU| 2961|
+-------+-----+
only showing top 20 rows



In [90]:
result_affiliation.join(organization, result_affiliation["organization"]==organization["id"]).count()

110239135

## 2.2. Dataframe `organization_originalid` (21 M)

Links organization id in the OpenAIRE graph with original id in the original data source?

Table structure:

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>originalid</mark> | string | Unique identifier of the linked organization in the original data source?|

In [88]:
organization_originalid = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/organization_originalid").cache()
print('Number of organization_originalid available:', organization_originalid.count())
print('Number of organization_originalid without duplicates:', organization_originalid.dropDuplicates().count())
print('Number of organization_originalid without duplicated ids:', organization_originalid.dropDuplicates(["id"]).count())
organization_originalid.printSchema()
organization_originalid.show(n=10, truncate=120, vertical=True)

Number of organization_originalid available: 314059
Number of organization_originalid without duplicates: 314059
Number of organization_originalid without duplicated ids: 314059
root
 |-- id: string (nullable = true)
 |-- originalid: string (nullable = true)

-RECORD 0----------------------------------------------------
 id         | anr_________::1c325fed2fc40eeeeeffbfbc4f4e5f4d 
 originalid | anr_________::5342bc2dab6ca36a49e9e0eb3bef8f8f 
-RECORD 1----------------------------------------------------
 id         | anr_________::bb8d415ee418e98e058e936192ec53e8 
 originalid | anr_________::f31dd5b3fea339f606c5cadc47f0c8f1 
-RECORD 2----------------------------------------------------
 id         | corda_______::456f6887dd0fa8ad4a4e6919ea3c62ce 
 originalid | corda_______::998731013                        
-RECORD 3----------------------------------------------------
 id         | openorgs____::0fdd6fd8d2273c1be274b8d3c285bdea 
 originalid | openorgs____::0000099849                    

## 2.3. Dataframe `organization_pids` (16 M)

| Field Name | Type | Description | 
| --- | --- | --- |
| <mark>id</mark> | string | Unique identifier of the result in the OpenAIRE graph |
| <mark>type</mark> | string | Type of external id value |
| <mark>pid</mark> | string | External id value |

In [89]:
organization_pids = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/organization_pids").cache()
print('Number of organization_pids available:', organization_pids.count())
print('Number of organization_pids without duplicates:', organization_pids.dropDuplicates().count())
print('Number of organization_pids without duplicated ids:', organization_pids.dropDuplicates(["id"]).count())
organization_pids.printSchema()
organization_pids.show(n=10, truncate=120, vertical=True)

Number of organization_pids available: 829997


Number of organization_pids without duplicates: 454406
Number of organization_pids without duplicated ids: 180242
root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- pid: string (nullable = true)

-RECORD 0----------------------------------------------
 id   | anr_________::bb8d415ee418e98e058e936192ec53e8 
 type | RNSR                                           
 pid  | RNSR:200616946Y                                
-RECORD 1----------------------------------------------
 id   | corda_______::456f6887dd0fa8ad4a4e6919ea3c62ce 
 type | PIC                                            
 pid  | 998731013                                      
-RECORD 2----------------------------------------------
 id   | openorgs____::0fdd6fd8d2273c1be274b8d3c285bdea 
 type | Participant Identification Code                
 pid  | 938001738                                      
-RECORD 3----------------------------------------------
 id   | openorgs____::0fdd6fd8d2273c1be274b8d3c

In [93]:
df = organization_pids.groupBy("type").agg(F.count("*").alias("count"))
df.orderBy(df["count"].desc()).show(20)

+--------------------+------+
|                type| count|
+--------------------+------+
|                 ROR|210130|
|                GRID|207691|
|Participant Ident...|106364|
|International Sta...|101746|
|            Wikidata| 92605|
|              OrgRef| 31699|
|             FundRef| 30960|
|                RNSR| 23102|
|              OrgReg|  9527|
|                 PIC|  7389|
|                 NIF|  2817|
|            RingGold|  1843|
|                CNRS|  1672|
|Research Resource...|  1062|
|               UKPRN|   350|
|                HESA|   343|
|                UCAS|   312|
|               DGEEC|   207|
|Virtual Internati...|    66|
|                ISNI|    58|
+--------------------+------+
only showing top 20 rows



# 3. Tables associated with Authors

## 3.1. Dataframe `author_pid` (12 G)

Not very promising yet

In [94]:
author_pid = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/author_pid").cache()
print('Number of author_pid available:', author_pid.count())
print('Number of author_pid without duplicates:', author_pid.dropDuplicates().count())
print('Number of author_pid without duplicated ids:', author_pid.dropDuplicates(["id"]).count())
author_pid.printSchema()
author_pid.show(n=10, truncate=120, vertical=True)

Number of author_pid available: 387760707


Number of author_pid without duplicates: 371166528


Number of author_pid without duplicated ids: 106675250
root
 |-- id: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- pid: string (nullable = true)

-RECORD 0----------------------------------------------
 id   | core_ac_uk__::05422a8759d1dd9085f6b2d27bb4b9d0 
 rank | 1                                              
 type | Open Researcher and Contributor ID             
 pid  | 0000-0002-0157-7034                            
-RECORD 1----------------------------------------------
 id   | core_ac_uk__::3dd5bd00dde9c9c5e2c03b85b0c2307f 
 rank | 1                                              
 type | Open Researcher and Contributor ID             
 pid  | 0000-0002-5130-291x                            
-RECORD 2----------------------------------------------
 id   | core_ac_uk__::3e172c545a0449446dc32795791895b7 
 rank | 1                                              
 type | Open Researcher and Contributor ID             
 pid  | 0000-

# 4. Tables associated with Projects

## 4.1. Dataframe `project` (358 M)

In [96]:
project = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/project").cache()
print('Number of project available:', project.count())
print('Number of project without duplicates:', project.dropDuplicates().count())
print('Number of project without duplicated ids:', project.dropDuplicates(["id"]).count())
project.printSchema()
project.show(n=10, truncate=120, vertical=True)

Number of project available: 3295519


Number of project without duplicates: 3295519


Number of project without duplicated ids: 3295519
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- acronym: string (nullable = true)
 |-- code: string (nullable = true)
 |-- callidentifier: string (nullable = true)
 |-- startdate: string (nullable = true)
 |-- enddate: string (nullable = true)
 |-- totalcost: float (nullable = true)
 |-- currency: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------
 id             | snsf________::b9a1d494f88facd686cc85af93a6f51e                                                                           
 title          | RAinRARE - Multilevel analyses of retinoic acid signaling to understand and treat a rare form of progressive motor im... 
 acronym        | null                                                                                                                     
 code           | 185656        

## 4.2. Dataframe `project_funding` (131 M)

In [97]:
project_funding = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/project_funding").cache()
print('Number of project_funding available:', project_funding.count())
print('Number of project_funding without duplicates:', project_funding.dropDuplicates().count())
print('Number of project_funding without duplicated ids:', project_funding.dropDuplicates(["id"]).count())
project_funding.printSchema()
project_funding.show(n=10, truncate=120, vertical=True)

Number of project_funding available: 3295519


Number of project_funding without duplicates: 3295519


Number of project_funding without duplicated ids: 3295519
root
 |-- id: string (nullable = true)
 |-- funder: string (nullable = true)
 |-- funding_lvl0: string (nullable = true)
 |-- funding_lvl1: string (nullable = true)
 |-- funding_lvl2: string (nullable = true)

-RECORD 0-------------------------------------------------------------
 id           | snsf________::b9a1d494f88facd686cc85af93a6f51e        
 funder       | Swiss National Science Foundation                     
 funding_lvl0 | Programmes                                            
 funding_lvl1 | ERA-NET                                               
 funding_lvl2 | ERA-E-RARE                                            
-RECORD 1-------------------------------------------------------------
 id           | nih_________::bdd53acbb1d3f91c0c6d6984164f9190        
 funder       | National Institutes of Health                         
 funding_lvl0 | NATIONAL INSTITUTE ON AGING                           
 funding_lvl1 |       

## 4.3. Dataframe `project_subject` (11 M)

In [105]:
project_subject = spark.read.parquet("/export/ml4ds/IntelComp/Datalake/openaire/20230328/project_subject").cache()
print('Number of project_subject available:', project_subject.count())
print('Number of project_subject without duplicates:', project_subject.dropDuplicates().count())
print('Number of project_subject without duplicated ids:', project_subject.dropDuplicates(["id"]).count())
project_subject.printSchema()
project_subject.show(n=10, truncate=120, vertical=True)

Number of project_subject available: 381026


Number of project_subject without duplicates: 381026
Number of project_subject without duplicated ids: 277739
root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- subject: string (nullable = true)

-RECORD 0--------------------------------------------------------------------------------------------
 id      | nsf_________::17a380395e0bcdcdf5c2d3e569655211                                            
 type    | Field of Application (NSF)                                                                
 subject | Other Applications NEC                                                                    
-RECORD 1--------------------------------------------------------------------------------------------
 id      | nsf_________::e23f853474e0f213ffe374324221f670                                            
 type    | Field of Application (NSF)                                                                
 subject | Mathematics                                       

23/07/04 03:43:43 ERROR TaskSchedulerImpl: Lost executor 44 on node21.cluster.tsc.uc3m.es: Executor finished with state LOST
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_39_144 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_1514_320 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_841_309 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_882_329 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_841_181 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_924_12 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_1514_168 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_1514_170 !
23/07/04 03:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_841_69 !
23/07/04 03